In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from tqdm import tqdm

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import json
qa_pairs = []
text = ""

with open('/content/training_data_multiarith.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list[:int(len(json_list)*0.7)]:
    result = json.loads(json_str)
    text_to_append = result['prompt'] + " " + result['completion']
    qa_pairs.append(text_to_append)

for question in qa_pairs:
  text += f"{question} {tokenizer.eos_token}"

In [ ]:
input_ids = tokenizer.encode(qa_pairs, add_special_tokens=True, return_tensors='pt')

In [ ]:
input_ids

tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 5

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(200):
  optimizer.zero_grad()
  loss = model(input_ids.to(device), labels=input_ids.to(device))[0]
  loss.backward()
  optimizer.step()
  scheduler.step()
  print(f"Epoch {epoch}, Loss {loss.item()}")

Epoch 0, Loss 11.654924392700195
Epoch 1, Loss 2.8704211711883545
Epoch 2, Loss 0.21187637746334076
Epoch 3, Loss 0.0905274897813797
Epoch 4, Loss 0.03149478882551193
Epoch 5, Loss 0.022214015945792198
Epoch 6, Loss 0.01889389380812645
Epoch 7, Loss 0.026254737749695778
Epoch 8, Loss 0.010652365162968636
Epoch 9, Loss 0.008659910410642624
Epoch 10, Loss 0.022755874320864677
Epoch 11, Loss 0.007373917382210493
Epoch 12, Loss 0.005954521708190441
Epoch 13, Loss 0.023145370185375214
Epoch 14, Loss 0.007304768543690443
Epoch 15, Loss 0.009630787186324596
Epoch 16, Loss 0.008733968250453472
Epoch 17, Loss 0.010440188460052013
Epoch 18, Loss 0.003905759658664465
Epoch 19, Loss 0.009268702939152718
Epoch 20, Loss 0.00826182309538126
Epoch 21, Loss 0.010271704755723476
Epoch 22, Loss 0.00700867036357522
Epoch 23, Loss 0.02441863901913166
Epoch 24, Loss 0.024193529039621353
Epoch 25, Loss 0.010371598415076733
Epoch 26, Loss 0.005928059108555317
Epoch 27, Loss 0.004236606415361166
Epoch 28, Loss

In [ ]:
model.eval()
question = "The school cafeteria ordered 42 red apples and 7 green apples for students lunches. But, if only 9 students wanted fruit, how many extra did the cafeteria end up with?"
input_id = tokenizer.encode(question, add_special_tokens=False, return_tensors='pt').to(device)
# Generate the answer using the model
sample_output = model.generate(input_id, do_sample=True, max_length=200, temperature = 0.7)

# Decode the generated answer using the tokenizer
answer = tokenizer.decode(sample_output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2ForQuestionAnswering, AdamW
from torch.utils.data import DataLoader, Dataset

# Define your custom questions and answers
questions = ["What is the capital of France?", "Who wrote Hamlet?"]
answers = ["Paris", "William Shakespeare"]

# Create a custom dataset
class CustomQADataset(Dataset):
    def __init__(self, tokenizer, questions, answers):
        self.tokenizer = tokenizer
        self.questions = questions
        self.answers = answers
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add padding token

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        print("Index:", idx)
        question = self.questions[idx]
        answer = self.answers[idx]

        # Tokenize question and answer
        question_encoding = self.tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=1024  # Adjust max length as per your requirements
        )
        answer_encoding = self.tokenizer.encode(
            answer,
            add_special_tokens=False,
            truncation=True,
            max_length=1024  # Adjust max length as per your requirements
        )

        return {
            "input_ids": question_encoding["input_ids"].squeeze(),
            "attention_mask": question_encoding["attention_mask"].squeeze(),
            "start_positions": torch.tensor([answer_encoding[0]]),
            "end_positions": torch.tensor([answer_encoding[-1]])
        }

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2ForQuestionAnswering.from_pretrained("gpt2")

# Fine-tuning parameters
batch_size = 8
lr = 5e-5
epochs = 3

# Prepare data loader
dataset = CustomQADataset(tokenizer, questions, answers)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Prepare optimizer
optimizer = AdamW(model.parameters(), lr=lr)

# Fine-tuning loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        start_positions = batch["start_positions"]
        end_positions = batch["end_positions"]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions
        )

        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2_qa_model")

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Index: 0
Index: 1


IndexError: index out of range in self